In [12]:
from azure.identity import AzureCliCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, ChatCompletionAgent
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from dotenv import load_dotenv
import asyncio
import requests
from semantic_kernel.functions import kernel_function

load_dotenv()

model = os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME")

project_client = AzureAIAgent.create_client(credential=AzureCliCredential(),
                           conn_str=os.getenv("PROJECT_CONNECTION_STRING")
)


class RecognizedRevenue:
    # def __init__(self, api_key: str):
    #     self.api_key = api_key

    @kernel_function(description="Gets recognized revenue for single or multiple companies.")
    def get_recognized_revenue(self, companyName:str):
        if companyName.lower() == "all":
            return {"Company A":"$220000", "Company B":"$150000", "Company C":"$180000"}
        if companyName.lower() == "company a":
            return "$220000"
        if companyName.lower() == "company b":
            return "$150000"
        if companyName.lower() == "company c":
            return "$180000"

class HistoricalPayment:
    # def __init__(self, api_key: str):
    #     self.api_key = api_key

    @kernel_function(description="Gets historical payment 'on-time' patterns.")
    def get_historical_payment_patterns(self, companyName:str):
        if companyName.lower() == "all":
            return {"Company A":"95%", "Company B":"85%", "Company C":"60%"}
        if companyName.lower() == "company a":
            return "95%"
        if companyName.lower() == "company b":
            return "85%"
        if companyName.lower() == "company c":
            return "60%"

class CashFlow:
    # def __init__(self, api_key: str):
    #     self.api_key = api_key

    @kernel_function(description="Calculates cash flow.")
    def cash_flow_calculation(self, inflow:int, outflow:int, riskyCustomerRevenue:int):
        return inflow-outflow-riskyCustomerRevenue
        
agent_definition = await project_client.agents.create_agent(
        model=model,
        name="Data_Access_Agent",
        instructions="""
        You are a specialized AI agent responsible for providing access to financial data upon request
        by other agents in the system. You hold complete and accurate data for payment history and revenue recognition. 
        Other agents depend on you to retrieve or analyze data. If recognized revenue is required, call 'RecognizedRevenue()'
        and if historical payment 'on-time' pattern is required, call 'HistoricalPayment()'. If any company has on-time payment
        less than '70%' then print 'RiskyCustomer:customer_name'. Also the total outflow is $90000. If cash flow calculation is needed
        then call 'CashFlow()' (If there is no risky customer send 0 for riskyCustomerRevenue parameter).
        """,
    )

agent = AzureAIAgent(
        client=project_client,
        definition=agent_definition,
        plugins=[RecognizedRevenue(), HistoricalPayment(), CashFlow()],
    )


In [ ]:

thread = project_client.agents.get_thread(thread_id="thread_ZPwygyzy2n7vgGg8F1t9fQLu")
message = project_client.agents.create_message(thread_id="thread_ZPwygyzy2n7vgGg8F1t9fQLu")
async def get_response_from_agent():
    response =  await agent.get_response(
        messages = user_input,
    )
    
    return response

while(True):
    user_input = input()
    if(user_input=="end" or "END"):
        break
    response = await get_response_from_agent()
    print(response)
   


C:\Users\Avik\AppData\Local\Temp\ipykernel_66504\3481893850.py:1: RuntimeWarning: coroutine 'AgentsOperations.create_thread' was never awaited
  thread = project_client.agents.get_thread(thread_id="thread_ZPwygyzy2n7vgGg8F1t9fQLu")


To calculate cash flow, I need recognized revenue and historical payment 'on-time' patterns for various companies. Which companies would you like me to evaluate, or would you like this done for all available companies?
To provide the requested financial insights, please specify the name of the company or companies for which the data is required.
Please specify your query or request concerning financial data (e.g., recognized revenue, historical payment patterns, or cash flow calculation). Let me know how I can assist.
The total cash flow is $280,000.
Cash flow is calculated by subtracting the total outflow from the total inflow during a specific period. It reflects the net flow of cash in and out of an organization. Here's the formula:

**Cash Flow = Inflow - Outflow**

If there are recognized revenues from "risky customers," these may impact the inflow adjustments, and we'd include this data. For companies with an on-time payment percentage of less than 70%, we might flag them as risk